<a href="https://colab.research.google.com/github/prashantmalan/Chatbot/blob/main/XmltoDf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import xml.etree.ElementTree as ET
import pandas as pd
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Assuming the XML file is named 'data.xml'
tree = ET.parse('/content/drive/My Drive/Colab Notebooks/anomaly/ird-ex01-vanilla-swap.xml')
root = tree.getroot()

def get_data_for_xpaths(element, current_path="", data=None, counter=None):
  """
  Recursively extracts data for each XPath, handling duplicates with a global counter.

  Args:
    element: The current XML element.
    current_path: The XPath expression leading to the current element.
    data: A dictionary to store the extracted data.
    counter: A dictionary to keep track of duplicate paths.

  Returns:
    A dictionary where keys are unique XPaths and values are data.
  """
  if data is None:
    data = {}
  if counter is None:
    counter = {}
  if not current_path:
    current_path = element.tag

  attrib_str = ""
  if element.attrib:
    attrib_str = " ".join([f"@{k}='{v}'" for k, v in element.attrib.items()])
  path_with_attrib = f"{current_path} [{attrib_str}]"

  if path_with_attrib not in counter:
    counter[path_with_attrib] = 0
  else:
    counter[path_with_attrib] += 1
  unique_path = f"{path_with_attrib}{counter[path_with_attrib]}" if counter[path_with_attrib] > 0 else path_with_attrib

  data[unique_path] = element.text

  for child in element:
    child_path = f"{unique_path}/{child.tag}"
    get_data_for_xpaths(child, child_path, data, counter)  # Pass the same counter down
  return data

# Get data for XPaths
data = get_data_for_xpaths(root)

# Create DataFrame
df = pd.DataFrame(data, index=[0])
df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,{http://www.fpml.org/FpML-5/transparency}publicExecutionReport [@fpmlVersion='5-2' @{http://www.w3.org/2001/XMLSchema-instance}schemaLocation='http://www.fpml.org/FpML-5/transparency ../../fpml-main-5-2.xsd'],{http://www.fpml.org/FpML-5/transparency}publicExecutionReport [@fpmlVersion='5-2' @{http://www.w3.org/2001/XMLSchema-instance}schemaLocation='http://www.fpml.org/FpML-5/transparency ../../fpml-main-5-2.xsd']/{http://www.fpml.org/FpML-5/transparency}header [],{http://www.fpml.org/FpML-5/transparency}publicExecutionReport [@fpmlVersion='5-2' @{http://www.w3.org/2001/XMLSchema-instance}schemaLocation='http://www.fpml.org/FpML-5/transparency ../../fpml-main-5-2.xsd']/{http://www.fpml.org/FpML-5/transparency}header []/{http://www.fpml.org/FpML-5/transparency}messageId [@messageIdScheme='http://abc.com/message-id'],{http://www.fpml.org/FpML-5/transparency}publicExecutionReport [@fpmlVersion='5-2' @{http://www.w3.org/2001/XMLSchema-instance}schemaLocation='http://www.fpml.org/FpML-5/transparency ../../fpml-main-5-2.xsd']/{http://www.fpml.org/FpML-5/transparency}header []/{http://www.fpml.org/FpML-5/transparency}sentBy [],{http://www.fpml.org/FpML-5/transparency}publicExecutionReport [@fpmlVersion='5-2' @{http://www.w3.org/2001/XMLSchema-instance}schemaLocation='http://www.fpml.org/FpML-5/transparency ../../fpml-main-5-2.xsd']/{http://www.fpml.org/FpML-5/transparency}header []/{http://www.fpml.org/FpML-5/transparency}sendTo [],{http://www.fpml.org/FpML-5/transparency}publicExecutionReport [@fpmlVersion='5-2' @{http://www.w3.org/2001/XMLSchema-instance}schemaLocation='http://www.fpml.org/FpML-5/transparency ../../fpml-main-5-2.xsd']/{http://www.fpml.org/FpML-5/transparency}header []/{http://www.fpml.org/FpML-5/transparency}creationTimestamp [],{http://www.fpml.org/FpML-5/transparency}publicExecutionReport [@fpmlVersion='5-2' @{http://www.w3.org/2001/XMLSchema-instance}schemaLocation='http://www.fpml.org/FpML-5/transparency ../../fpml-main-5-2.xsd']/{http://www.fpml.org/FpML-5/transparency}isCorrection [],{http://www.fpml.org/FpML-5/transparency}publicExecutionReport [@fpmlVersion='5-2' @{http://www.w3.org/2001/XMLSchema-instance}schemaLocation='http://www.fpml.org/FpML-5/transparency ../../fpml-main-5-2.xsd']/{http://www.fpml.org/FpML-5/transparency}correlationId [@correlationIdScheme='http://abc.com/correlation-id'],{http://www.fpml.org/FpML-5/transparency}publicExecutionReport [@fpmlVersion='5-2' @{http://www.w3.org/2001/XMLSchema-instance}schemaLocation='http://www.fpml.org/FpML-5/transparency ../../fpml-main-5-2.xsd']/{http://www.fpml.org/FpML-5/transparency}sequenceNumber [],{http://www.fpml.org/FpML-5/transparency}publicExecutionReport [@fpmlVersion='5-2' @{http://www.w3.org/2001/XMLSchema-instance}schemaLocation='http://www.fpml.org/FpML-5/transparency ../../fpml-main-5-2.xsd']/{http://www.fpml.org/FpML-5/transparency}originatingEvent [],...,{http://www.fpml.org/FpML-5/transparency}publicExecutionReport [@fpmlVersion='5-2' @{http://www.w3.org/2001/XMLSchema-instance}schemaLocation='http://www.fpml.org/FpML-5/transparency ../../fpml-main-5-2.xsd']/{http://www.fpml.org/FpML-5/transparency}trade []/{http://www.fpml.org/FpML-5/transparency}swap []/{http://www.fpml.org/FpML-5/transparency}swapStream []1/{http://www.fpml.org/FpML-5/transparency}paymentDates []/{http://www.fpml.org/FpML-5/transparency}paymentFrequency []/{http://www.fpml.org/FpML-5/transparency}period [],{http://www.fpml.org/FpML-5/transparency}publicExecutionReport [@fpmlVersion='5-2' @{http://www.w3.org/2001/XMLSchema-instance}schemaLocation='http://www.fpml.org/FpML-5/transparency ../../fpml-main-5-2.xsd']/{http://www.fpml.org/FpML-5/transparency}trade []/{http://www.fpml.org/FpML-5/transparency}swap []/{http://www.fpml.org/FpML-5/transparency}swapStream []1/{http://www.fpml.org/FpML-5/transparency}calculationPeriodAmount [],{http://www.fpml.org/FpML-5/transparency}publicExecutionReport [@fpmlVersion='5-2' @{http://www.w3.org/2001/XMLSchema-instance}schemaLoc

In [8]:
# prompt: from the above xml  I want to capture all the xpath and fielname in 2 columns , I want to ignore the tags which starts from urn:

import xml.etree.ElementTree as ET
import pandas as pd

def extract_xpath_filename(element, current_path="", data=None):
  """
  Recursively extracts XPath and filename from an XML element, ignoring 'urn:' tags.

  Args:
    element: The current XML element.
    current_path: The XPath expression leading to the current element.
    data: A list to store the extracted data (XPath, filename).

  Returns:
    A list of tuples (XPath, filename).
  """
  if data is None:
    data = []

  if not current_path:
    current_path = element.tag

  if not element.tag.startswith('urn:'):
    if element.text:
      data.append((current_path, element.text))

  for child in element:
    child_path = f"{current_path}/{child.tag}"
    extract_xpath_filename(child, child_path, data)

  return data

# Get data for XPaths and filenames
data_list = extract_xpath_filename(root)

# Create DataFrame
df = pd.DataFrame(data_list, columns=["XPath", "Filename"])
df


NameError: name 'root' is not defined

In [11]:
import xml.etree.ElementTree as ET
import pandas as pd
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Assuming the XML file is named 'data.xml'
tree = ET.parse('/content/drive/My Drive/Colab Notebooks/anomaly/WrapperSample_NEWT_COMM_MultipleEAs_Feb14.xml')
root = tree.getroot() # define root

def extract_xpath_filename(element, current_path="", data=None):
  """
  Recursively extracts XPath and filename from an XML element, ignoring 'urn:' tags.

  Args:
    element: The current XML element.
    current_path: The XPath expression leading to the current element.
    data: A list to store the extracted data (XPath, filename).

  Returns:
    A list of tuples (XPath, filename).
  """
  if data is None:
    data = []

  if not current_path:
    current_path = element.tag

  if not element.tag.startswith('urn:'):
    if element.text:
      data.append((current_path, element.text))

  for child in element:
    child_path = f"{current_path}/{child.tag}"
    extract_xpath_filename(child, child_path, data)

  return data

# Get data for XPaths and filenames
data_list = extract_xpath_filename(root)

# Create DataFrame
df = pd.DataFrame(data_list, columns=["XPath", "Filename"])
df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,XPath,Filename
0,{DTCC_Wrapper}DTCC_Wrapper,\n\t
1,{DTCC_Wrapper}DTCC_Wrapper/{DTCC_Wrapper}Execu...,\n\t\t
2,{DTCC_Wrapper}DTCC_Wrapper/{DTCC_Wrapper}Execu...,\n\t\t\t
3,{DTCC_Wrapper}DTCC_Wrapper/{DTCC_Wrapper}Execu...,HARNESSAUTOUTIESMA10ALLTESTCOMMNEWTALLASSTLT02
4,{DTCC_Wrapper}DTCC_Wrapper/{DTCC_Wrapper}Execu...,HARNESSAUTOUTIESMA10ALLTESTCOMMNEWTALLASSTLT01
...,...,...
675,{DTCC_Wrapper}DTCC_Wrapper/{urn:iso:std:iso:20...,PACKAGEID1234
676,{DTCC_Wrapper}DTCC_Wrapper/{urn:iso:std:iso:20...,\n\t\t\t\t\t\t\t\t\t\t\t\t
677,{DTCC_Wrapper}DTCC_Wrapper/{urn:iso:std:iso:20...,\n\t\t\t\t\t\t\t\t\t\t\t\t\t
678,{DTCC_Wrapper}DTCC_Wrapper/{urn:iso:std:iso:20...,23.567


In [13]:
import xml.etree.ElementTree as ET
import pandas as pd
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Assuming the XML file is named 'data.xml'
tree = ET.parse('/content/drive/My Drive/Colab Notebooks/anomaly/WrapperSample_NEWT_COMM_MultipleEAs_Feb14.xml')
root = tree.getroot()

def extract_xpath_fieldname(element, current_path="", data=None):
  """
  Recursively extracts XPath and fieldname from an XML element, ignoring 'urn:' tags
  and omitting data values.

  Args:
    element: The current XML element.
    current_path: The XPath expression leading to the current element.
    data: A list to store the extracted data (XPath, fieldname).

  Returns:
    A list of tuples (XPath, fieldname).
  """
  if data is None:
    data = []

  if not current_path:
    current_path = element.tag

  if not element.tag.startswith('urn:'):
      data.append((current_path, element.tag)) # Add fieldname (element tag)

  for child in element:
    child_path = f"{current_path}/{child.tag}"
    extract_xpath_fieldname(child, child_path, data)

  return data

# Get XPath and fieldname data
data_list = extract_xpath_fieldname(root)

# Create DataFrame
df = pd.DataFrame(data_list, columns=["XPath", "Fieldname"])
df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,XPath,Fieldname
0,{DTCC_Wrapper}DTCC_Wrapper,{DTCC_Wrapper}DTCC_Wrapper
1,{DTCC_Wrapper}DTCC_Wrapper/{DTCC_Wrapper}Execu...,{DTCC_Wrapper}ExecutionAgentBlock
2,{DTCC_Wrapper}DTCC_Wrapper/{DTCC_Wrapper}Execu...,{DTCC_Wrapper}ExecutionAgent
3,{DTCC_Wrapper}DTCC_Wrapper/{DTCC_Wrapper}Execu...,{DTCC_Wrapper}Uid
4,{DTCC_Wrapper}DTCC_Wrapper/{DTCC_Wrapper}Execu...,{DTCC_Wrapper}Uid
...,...,...
675,{DTCC_Wrapper}DTCC_Wrapper/{urn:iso:std:iso:20...,{urn:iso:std:iso:20022:tech:xsd:auth.030.001.0...
676,{DTCC_Wrapper}DTCC_Wrapper/{urn:iso:std:iso:20...,{urn:iso:std:iso:20022:tech:xsd:auth.030.001.0...
677,{DTCC_Wrapper}DTCC_Wrapper/{urn:iso:std:iso:20...,{urn:iso:std:iso:20022:tech:xsd:auth.030.001.0...
678,{DTCC_Wrapper}DTCC_Wrapper/{urn:iso:std:iso:20...,{urn:iso:std:iso:20022:tech:xsd:auth.030.001.0...


In [16]:
import xml.etree.ElementTree as ET
import pandas as pd
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Assuming the XML file is named 'data.xml'
tree = ET.parse('/content/drive/My Drive/Colab Notebooks/anomaly/WrapperSample_NEWT_COMM_MultipleEAs_Feb14.xml')
root = tree.getroot()

def extract_xpath_fieldname(element, current_path="", data=None):
  """
  Recursively extracts XPath and fieldname from an XML element, ignoring 'urn:' tags
  and omitting data values.

  Args:
    element: The current XML element.
    current_path: The XPath expression leading to the current element.
    data: A list to store the extracted data (XPath, fieldname).

  Returns:
    A list of tuples (XPath, fieldname).
  """
  if data is None:
    data = []

  if not current_path:
    current_path = element.tag

  if not element.tag.startswith('urn:'):
    if not current_path.startswith('urn:'): # Check if current path starts with 'urn:'
      data.append((current_path, element.tag))

    for child in element:
      if not child.tag.startswith('urn:'): # Check child tag before proceeding
        child_path = f"{current_path}/{child.tag}"
        extract_xpath_fieldname(child, child_path, data)

  return data

# Get XPath and fieldname data
data_list = extract_xpath_fieldname(root)

# Create DataFrame
df = pd.DataFrame(data_list, columns=["XPath", "Fieldname"])
df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,XPath,Fieldname
0,{DTCC_Wrapper}DTCC_Wrapper,{DTCC_Wrapper}DTCC_Wrapper
1,{DTCC_Wrapper}DTCC_Wrapper/{DTCC_Wrapper}Execu...,{DTCC_Wrapper}ExecutionAgentBlock
2,{DTCC_Wrapper}DTCC_Wrapper/{DTCC_Wrapper}Execu...,{DTCC_Wrapper}ExecutionAgent
3,{DTCC_Wrapper}DTCC_Wrapper/{DTCC_Wrapper}Execu...,{DTCC_Wrapper}Uid
4,{DTCC_Wrapper}DTCC_Wrapper/{DTCC_Wrapper}Execu...,{DTCC_Wrapper}Uid
...,...,...
675,{DTCC_Wrapper}DTCC_Wrapper/{urn:iso:std:iso:20...,{urn:iso:std:iso:20022:tech:xsd:auth.030.001.0...
676,{DTCC_Wrapper}DTCC_Wrapper/{urn:iso:std:iso:20...,{urn:iso:std:iso:20022:tech:xsd:auth.030.001.0...
677,{DTCC_Wrapper}DTCC_Wrapper/{urn:iso:std:iso:20...,{urn:iso:std:iso:20022:tech:xsd:auth.030.001.0...
678,{DTCC_Wrapper}DTCC_Wrapper/{urn:iso:std:iso:20...,{urn:iso:std:iso:20022:tech:xsd:auth.030.001.0...


In [18]:
import xml.etree.ElementTree as ET
import pandas as pd
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Assuming the XML file is named 'data.xml'
tree = ET.parse('/content/drive/My Drive/Colab Notebooks/anomaly/WrapperSample_NEWT_COMM_MultipleEAs_Feb14.xml')
root = tree.getroot()

def extract_xpath_fieldname(element, current_path="", data=None):
  """
  Recursively extracts XPath and fieldname from an XML element, ignoring 'urn:' tags
  and omitting data values.

  Args:
    element: The current XML element.
    current_path: The XPath expression leading to the current element.
    data: A list to store the extracted data (XPath, fieldname).

  Returns:
    A list of tuples (XPath, fieldname).
  """
  if data is None:
    data = []

  if not current_path:
    current_path = element.tag

  if not element.tag.startswith('urn:'):
    if not any(part.startswith('urn:') for part in current_path.split('/')): # Check all parts of the path
      data.append((current_path, element.tag))

    for child in element:
      if not child.tag.startswith('urn:'):
        child_path = f"{current_path}/{child.tag}"
        extract_xpath_fieldname(child, child_path, data)

  return data

# Get XPath and fieldname data
data_list = extract_xpath_fieldname(root)

# Create DataFrame
df = pd.DataFrame(data_list, columns=["XPath", "Fieldname"])
df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,XPath,Fieldname
0,{DTCC_Wrapper}DTCC_Wrapper,{DTCC_Wrapper}DTCC_Wrapper
1,{DTCC_Wrapper}DTCC_Wrapper/{DTCC_Wrapper}Execu...,{DTCC_Wrapper}ExecutionAgentBlock
2,{DTCC_Wrapper}DTCC_Wrapper/{DTCC_Wrapper}Execu...,{DTCC_Wrapper}ExecutionAgent
3,{DTCC_Wrapper}DTCC_Wrapper/{DTCC_Wrapper}Execu...,{DTCC_Wrapper}Uid
4,{DTCC_Wrapper}DTCC_Wrapper/{DTCC_Wrapper}Execu...,{DTCC_Wrapper}Uid
...,...,...
675,{DTCC_Wrapper}DTCC_Wrapper/{urn:iso:std:iso:20...,{urn:iso:std:iso:20022:tech:xsd:auth.030.001.0...
676,{DTCC_Wrapper}DTCC_Wrapper/{urn:iso:std:iso:20...,{urn:iso:std:iso:20022:tech:xsd:auth.030.001.0...
677,{DTCC_Wrapper}DTCC_Wrapper/{urn:iso:std:iso:20...,{urn:iso:std:iso:20022:tech:xsd:auth.030.001.0...
678,{DTCC_Wrapper}DTCC_Wrapper/{urn:iso:std:iso:20...,{urn:iso:std:iso:20022:tech:xsd:auth.030.001.0...


In [22]:
import xml.etree.ElementTree as ET
import pandas as pd
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Assuming the XML file is named 'WrapperSample_NEWT_COMM_MultipleEAs_Feb14.xml'
tree = ET.parse('/content/drive/My Drive/Colab Notebooks/anomaly/WrapperSample_NEWT_COMM_MultipleEAs_Feb14.xml')
root = tree.getroot()

def extract_xpath_fieldname(element, current_path="", data=None):
  """
  Recursively extracts XPath and fieldname from an XML element, ignoring 'urn:' tags
  and omitting data values.

  Args:
    element: The current XML element.
    current_path: The XPath expression leading to the current element.
    data: A list to store the extracted data (XPath, fieldname).

  Returns:
    A list of tuples (XPath, fieldname).
  """
  if data is None:
    data = []

  if not current_path:
    current_path = element.tag

  if not element.tag.startswith('urn:'):
    path_parts = current_path.split('/')
    filtered_path = '/'.join([part for part in path_parts if not part.startswith('urn:')]) # Rebuild path without 'urn:' parts

    if filtered_path:
      data.append((filtered_path, element.tag))

    for child in element:
      if not child.tag.startswith('urn:'):
        child_path = f"{filtered_path}/{child.tag}"
        extract_xpath_fieldname(child, child_path, data)

  return data

# Get XPath and fieldname data
data_list = extract_xpath_fieldname(root)

# Create DataFrame
df = pd.DataFrame(data_list, columns=["XPath", "Fieldname"])
df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,XPath,Fieldname
0,{DTCC_Wrapper}DTCC_Wrapper,{DTCC_Wrapper}DTCC_Wrapper
1,{DTCC_Wrapper}DTCC_Wrapper/{DTCC_Wrapper}Execu...,{DTCC_Wrapper}ExecutionAgentBlock
2,{DTCC_Wrapper}DTCC_Wrapper/{DTCC_Wrapper}Execu...,{DTCC_Wrapper}ExecutionAgent
3,{DTCC_Wrapper}DTCC_Wrapper/{DTCC_Wrapper}Execu...,{DTCC_Wrapper}Uid
4,{DTCC_Wrapper}DTCC_Wrapper/{DTCC_Wrapper}Execu...,{DTCC_Wrapper}Uid
...,...,...
675,{DTCC_Wrapper}DTCC_Wrapper/{urn:iso:std:iso:20...,{urn:iso:std:iso:20022:tech:xsd:auth.030.001.0...
676,{DTCC_Wrapper}DTCC_Wrapper/{urn:iso:std:iso:20...,{urn:iso:std:iso:20022:tech:xsd:auth.030.001.0...
677,{DTCC_Wrapper}DTCC_Wrapper/{urn:iso:std:iso:20...,{urn:iso:std:iso:20022:tech:xsd:auth.030.001.0...
678,{DTCC_Wrapper}DTCC_Wrapper/{urn:iso:std:iso:20...,{urn:iso:std:iso:20022:tech:xsd:auth.030.001.0...


In [24]:
import xml.etree.ElementTree as ET
import pandas as pd
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Assuming the XML file is named 'WrapperSample_NEWT_COMM_MultipleEAs_Feb14.xml'
tree = ET.parse('/content/drive/My Drive/Colab Notebooks/anomaly/WrapperSample_NEWT_COMM_MultipleEAs_Feb14.xml')
root = tree.getroot()

def extract_xpath_fieldname(element, current_path="", data=None):
  """
  Recursively extracts XPath and fieldname from an XML element, ignoring 'urn:' tags
  and omitting data values.

  Args:
    element: The current XML element.
    current_path: The XPath expression leading to the current element.
    data: A list to store the extracted data (XPath, fieldname).

  Returns:
    A list of tuples (XPath, fieldname).
  """
  if data is None:
    data = []

  if not element.tag.startswith('urn:'): # Check element tag before processing
    if not current_path:
      current_path = element.tag

    path_parts = current_path.split('/')
    filtered_path = '/'.join([part for part in path_parts if not part.startswith('urn:')])

    if filtered_path:
      data.append((filtered_path, element.tag))

    for child in element:
      if not child.tag.startswith('urn:'):
        child_path = f"{filtered_path}/{child.tag}"
        extract_xpath_fieldname(child, child_path, data)

  return data

# Get XPath and fieldname data
data_list = extract_xpath_fieldname(root)

# Create DataFrame
df = pd.DataFrame(data_list, columns=["XPath", "Fieldname"])
df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,XPath,Fieldname
0,{DTCC_Wrapper}DTCC_Wrapper,{DTCC_Wrapper}DTCC_Wrapper
1,{DTCC_Wrapper}DTCC_Wrapper/{DTCC_Wrapper}Execu...,{DTCC_Wrapper}ExecutionAgentBlock
2,{DTCC_Wrapper}DTCC_Wrapper/{DTCC_Wrapper}Execu...,{DTCC_Wrapper}ExecutionAgent
3,{DTCC_Wrapper}DTCC_Wrapper/{DTCC_Wrapper}Execu...,{DTCC_Wrapper}Uid
4,{DTCC_Wrapper}DTCC_Wrapper/{DTCC_Wrapper}Execu...,{DTCC_Wrapper}Uid
...,...,...
675,{DTCC_Wrapper}DTCC_Wrapper/{urn:iso:std:iso:20...,{urn:iso:std:iso:20022:tech:xsd:auth.030.001.0...
676,{DTCC_Wrapper}DTCC_Wrapper/{urn:iso:std:iso:20...,{urn:iso:std:iso:20022:tech:xsd:auth.030.001.0...
677,{DTCC_Wrapper}DTCC_Wrapper/{urn:iso:std:iso:20...,{urn:iso:std:iso:20022:tech:xsd:auth.030.001.0...
678,{DTCC_Wrapper}DTCC_Wrapper/{urn:iso:std:iso:20...,{urn:iso:std:iso:20022:tech:xsd:auth.030.001.0...


In [29]:
import xml.etree.ElementTree as ET
import pandas as pd
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Assuming the XML file is named 'WrapperSample_NEWT_COMM_MultipleEAs_Feb14.xml'
tree = ET.parse('/content/drive/My Drive/Colab Notebooks/anomaly/WrapperSample_NEWT_COMM_MultipleEAs_Feb14.xml')
root = tree.getroot()

def extract_xpath_fieldname(element, current_path="", data=None):
  """
  Recursively extracts XPath and fieldname from an XML element,
  replacing tags containing 'urn:' with ''.

  Args:
    element: The current XML element.
    current_path: The XPath expression leading to the current element.
    data: A list to store the extracted data (XPath, fieldname).

  Returns:
    A list of tuples (XPath, fieldname).
  """
  if data is None:
    data = []

  if not current_path:
    current_path = element.tag

  # Replace tag if it contains 'urn:'
  tag = element.tag.replace("{urn:", "{").replace("}", "")

  path_parts = current_path.split('/')
  filtered_path = '/'.join([part.replace("{urn:", "{").replace("}", "") for part in path_parts])


  if filtered_path:
    data.append((filtered_path, tag))

  for child in element:
    child_path = f"{filtered_path}/{tag}"
    extract_xpath_fieldname(child, child_path, data)

  return data

# Get XPath and fieldname data
data_list = extract_xpath_fieldname(root)

# Create DataFrame
df = pd.DataFrame(data_list, columns=["XPath", "Fieldname"])
df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,XPath,Fieldname
0,{DTCC_WrapperDTCC_Wrapper,{DTCC_WrapperDTCC_Wrapper
1,{DTCC_WrapperDTCC_Wrapper/{DTCC_WrapperDTCC_Wr...,{DTCC_WrapperExecutionAgentBlock
2,{DTCC_WrapperDTCC_Wrapper/{DTCC_WrapperDTCC_Wr...,{DTCC_WrapperExecutionAgent
3,{DTCC_WrapperDTCC_Wrapper/{DTCC_WrapperDTCC_Wr...,{DTCC_WrapperUid
4,{DTCC_WrapperDTCC_Wrapper/{DTCC_WrapperDTCC_Wr...,{DTCC_WrapperUid
...,...,...
675,{DTCC_WrapperDTCC_Wrapper/{DTCC_WrapperDTCC_Wr...,{iso:std:iso:20022:tech:xsd:auth.030.001.03Cmp...
676,{DTCC_WrapperDTCC_Wrapper/{DTCC_WrapperDTCC_Wr...,{iso:std:iso:20022:tech:xsd:auth.030.001.03Pric
677,{DTCC_WrapperDTCC_Wrapper/{DTCC_WrapperDTCC_Wr...,{iso:std:iso:20022:tech:xsd:auth.030.001.03Mnt...
678,{DTCC_WrapperDTCC_Wrapper/{DTCC_WrapperDTCC_Wr...,{iso:std:iso:20022:tech:xsd:auth.030.001.03Amt


In [33]:
import xml.etree.ElementTree as ET
import pandas as pd
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Assuming the XML file is named 'WrapperSample_NEWT_COMM_MultipleEAs_Feb14.xml'
tree = ET.parse('/content/drive/My Drive/Colab Notebooks/anomaly/WrapperSample_NEWT_COMM_MultipleEAs_Feb14.xml')
root = tree.getroot()

def extract_xpath_fieldname(element, current_path="", data=None):
  """
  Recursively extracts XPath and fieldname from an XML element,
  replacing tags containing 'urn:' or 'iso:' with '' and 'std:' with ''.

  Args:
    element: The current XML element.
    current_path: The XPath expression leading to the current element.
    data: A list to store the extracted data (XPath, fieldname).

  Returns:
    A list of tuples (XPath, fieldname).
  """
  if data is None:
    data = []

  if not current_path:
    current_path = element.tag

  # Replace tag if it contains 'urn:', 'iso:', or 'std:'
  tag = element.tag.replace("{urn:", "{").replace("}", "").replace("iso:", "").replace("std:", "")

  path_parts = current_path.split('/')
  filtered_path = '/'.join([part.replace("{urn:", "{").replace("}", "").replace("iso:", "").replace("std:", "") for part in path_parts])

  if filtered_path:
    data.append((filtered_path, tag))

  for child in element:
    child_path = f"{filtered_path}/{tag}"
    extract_xpath_fieldname(child, child_path, data)

  return data

# Get XPath and fieldname data
data_list = extract_xpath_fieldname(root)

# Create DataFrame
df = pd.DataFrame(data_list, columns=["XPath", "Fieldname"])
df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,XPath,Fieldname
0,{DTCC_WrapperDTCC_Wrapper,{DTCC_WrapperDTCC_Wrapper
1,{DTCC_WrapperDTCC_Wrapper/{DTCC_WrapperDTCC_Wr...,{DTCC_WrapperExecutionAgentBlock
2,{DTCC_WrapperDTCC_Wrapper/{DTCC_WrapperDTCC_Wr...,{DTCC_WrapperExecutionAgent
3,{DTCC_WrapperDTCC_Wrapper/{DTCC_WrapperDTCC_Wr...,{DTCC_WrapperUid
4,{DTCC_WrapperDTCC_Wrapper/{DTCC_WrapperDTCC_Wr...,{DTCC_WrapperUid
...,...,...
675,{DTCC_WrapperDTCC_Wrapper/{DTCC_WrapperDTCC_Wr...,{20022:tech:xsd:auth.030.001.03CmplxTradId
676,{DTCC_WrapperDTCC_Wrapper/{DTCC_WrapperDTCC_Wr...,{20022:tech:xsd:auth.030.001.03Pric
677,{DTCC_WrapperDTCC_Wrapper/{DTCC_WrapperDTCC_Wr...,{20022:tech:xsd:auth.030.001.03MntryVal
678,{DTCC_WrapperDTCC_Wrapper/{DTCC_WrapperDTCC_Wr...,{20022:tech:xsd:auth.030.001.03Amt


In [35]:
import xml.etree.ElementTree as ET
import pandas as pd
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Assuming the XML file is named 'WrapperSample_NEWT_COMM_MultipleEAs_Feb14.xml'
tree = ET.parse('/content/drive/My Drive/Colab Notebooks/anomaly/WrapperSample_NEWT_COMM_MultipleEAs_Feb14.xml')
root = tree.getroot()

def extract_xpath_fieldname(element, current_path="", data=None):
  """
  Recursively extracts XPath and fieldname from an XML element,
  replacing tags containing 'urn:' or 'iso:' with '' and 'std:' with ''.

  Args:
    element: The current XML element.
    current_path: The XPath expression leading to the current element.
    data: A list to store the extracted data (XPath, fieldname).

  Returns:
    A list of tuples (XPath, fieldname).
  """
  if data is None:
    data = []

  if not current_path:
    current_path = element.tag

  # Replace tag if it contains 'urn:', 'iso:', or 'std:'
  tag = element.tag.replace("{urn:", "{").replace("}", "").replace("iso:", "").replace("std:", "")

  path_parts = current_path.split('/')
  filtered_path = '/'.join([part.replace("{urn:", "{").replace("}", "").replace("iso:", "").replace("std:", "") for part in path_parts])

  if filtered_path:
    data.append((filtered_path, tag))

  for child in element:
    # Construct child_path correctly using current_path
    child_path = f"{current_path}/{child.tag}"
    extract_xpath_fieldname(child, child_path, data)

  return data

# Get XPath and fieldname data
data_list = extract_xpath_fieldname(root)

# Create DataFrame
df = pd.DataFrame(data_list, columns=["XPath", "Fieldname"])
df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,XPath,Fieldname
0,{DTCC_WrapperDTCC_Wrapper,{DTCC_WrapperDTCC_Wrapper
1,{DTCC_WrapperDTCC_Wrapper/{DTCC_WrapperExecuti...,{DTCC_WrapperExecutionAgentBlock
2,{DTCC_WrapperDTCC_Wrapper/{DTCC_WrapperExecuti...,{DTCC_WrapperExecutionAgent
3,{DTCC_WrapperDTCC_Wrapper/{DTCC_WrapperExecuti...,{DTCC_WrapperUid
4,{DTCC_WrapperDTCC_Wrapper/{DTCC_WrapperExecuti...,{DTCC_WrapperUid
...,...,...
675,{DTCC_WrapperDTCC_Wrapper/{20022:tech:xsd:head...,{20022:tech:xsd:auth.030.001.03CmplxTradId
676,{DTCC_WrapperDTCC_Wrapper/{20022:tech:xsd:head...,{20022:tech:xsd:auth.030.001.03Pric
677,{DTCC_WrapperDTCC_Wrapper/{20022:tech:xsd:head...,{20022:tech:xsd:auth.030.001.03MntryVal
678,{DTCC_WrapperDTCC_Wrapper/{20022:tech:xsd:head...,{20022:tech:xsd:auth.030.001.03Amt


In [48]:
# final take

import xml.etree.ElementTree as ET
import pandas as pd
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Assuming the XML file is named 'WrapperSample_NEWT_COMM_MultipleEAs_Feb14.xml'
file_path=  '/content/drive/My Drive/Colab Notebooks/anomaly/WrapperSample_NEWT_COMM_MultipleEAs_Feb14.xml'
import xml.etree.ElementTree as ET

class XMLParser:
    def __init__(self, file_path):
        self.file_path = file_path

    def parse(self):
        try:
            tree = ET.parse(self.file_path)
            return self._extract_data(tree.getroot())
        except (ET.ParseError, FileNotFoundError) as e:
            print(f"Error parsing XML: {e}")
            return []

    def _extract_data(self, root):
        data = []
        self._build_xpath(root, "", data)
        return data

    def _build_xpath(self, elem, current_path, data):
        # Get the tag name without namespace
        tag_name = elem.tag.split('}')[-1] if '}' in elem.tag else elem.tag

        # Create the new path
        if current_path == "":
            new_path = f"//*." + tag_name  # Start with //*
        else:
            new_path = f"{current_path}/*." + tag_name  # Replace / with /*

        field_name = tag_name  # Field name is the tag name

        # Append the current XPath and field name to the data
        data.append((new_path, field_name))

        # Recursively process child elements
        for child in elem:
            self._build_xpath(child, new_path, data)

#file_path = 'path_to_your_xml_file.xml'  # Replace with your XML file path
parser = XMLParser(file_path)
parsed_data = parser.parse()

# Print the results in the desired format
print("XPath\tField Name")
for xpath, field_name in parsed_data:
    print(f"{xpath}\t{field_name}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
XPath	Field Name
//*.DTCC_Wrapper	DTCC_Wrapper
//*.DTCC_Wrapper/*.ExecutionAgentBlock	ExecutionAgentBlock
//*.DTCC_Wrapper/*.ExecutionAgentBlock/*.ExecutionAgent	ExecutionAgent
//*.DTCC_Wrapper/*.ExecutionAgentBlock/*.ExecutionAgent/*.Uid	Uid
//*.DTCC_Wrapper/*.ExecutionAgentBlock/*.ExecutionAgent/*.Uid	Uid
//*.DTCC_Wrapper/*.ExecutionAgentBlock/*.ExecutionAgent	ExecutionAgent
//*.DTCC_Wrapper/*.ExecutionAgentBlock/*.ExecutionAgent/*.Uid	Uid
//*.DTCC_Wrapper/*.ExecutionAgentBlock/*.ExecutionAgent/*.Uid	Uid
//*.DTCC_Wrapper/*.BizData	BizData
//*.DTCC_Wrapper/*.BizData/*.Hdr	Hdr
//*.DTCC_Wrapper/*.BizData/*.Hdr/*.AppHdr	AppHdr
//*.DTCC_Wrapper/*.BizData/*.Hdr/*.AppHdr/*.Fr	Fr
//*.DTCC_Wrapper/*.BizData/*.Hdr/*.AppHdr/*.Fr/*.OrgId	OrgId
//*.DTCC_Wrapper/*.BizData/*.Hdr/*.AppHdr/*.Fr/*.OrgId/*.Id	Id
//*.DTCC_Wrapper/*.BizData/*.Hdr/*.AppHdr/*.Fr/*.OrgId/*.Id/*.Or